In [1]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

import datetime as dt
from datetime import datetime, timedelta
import calendar

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

from Position_Report_Calcs import Position_Reporting

In [2]:
plotly.offline.init_notebook_mode(connected=True)

In [3]:
def reporting_date(run_date='today'):
    if run_date=='today':
        current_date = dt.datetime.today().date()
        current_year = current_date.year
    else:
        current_date = run_date
        current_year = current_date.year
    return current_date, current_year
        
#current_date, current_year = reporting_date(run_date = dt.datetime.strptime('21-06-2024', '%d-%m-%Y').date())    # for a specific date
current_date, current_year = reporting_date()    # as at today

In [4]:
def get_month_year(months_fwd):   # This is incase the months fwd overlaps into a different year
    current_date = dt.datetime.today().date()
    month = current_date.month + months_fwd
    year = current_date.year
    if month > 12:
        month -=12
        year += 1
        
    _, last_day_of_month = calendar.monthrange(year, month)
    date_to_use = dt.date(year, month, last_day_of_month)
    return date_to_use

In [5]:
## Read in the position data
mkts = ['ACCU','NZU','EUA','OTHER']

positions = dict()
for m in mkts:
    positions[m] = pd.read_excel('Positions.xlsx', sheet_name=m)
    positions[m]['Expiry'] = pd.to_datetime(positions[m].Expiry).dt.date

premiums = pd.read_excel('Positions.xlsx', sheet_name='Index') 

In [6]:
## Define the dates we want to use for reporting
#current_date = dt.datetime.today().date()
#current_year = str(dt.datetime.today().date().year)

_, last_day = calendar.monthrange(current_date.year, current_date.month)
end_of_month = dt.date(current_date.year, current_date.month, last_day)
two_months = get_month_year(2)
three_months = get_month_year(3)
four_months = get_month_year(4)
five_months = get_month_year(5)
six_months = get_month_year(6)

dec_exp = dt.datetime.strptime('16-12-2025', '%d-%m-%Y').date()

## MORE GRANULAR ##
# Get the current date
today = datetime.now()
# Calculate the number of days until the next Saturday (because we want the full friday session)
days_until_saturday = (5 - today.weekday()) % 7  # Saturday is weekday 5
# Get the date of the end of the current week (this coming Saturday)
end_of_this_week_saturday = today + timedelta(days=days_until_saturday)
# Get the end of next week by adding 7 more days
end_this_wk = end_of_this_week_saturday.date()
four_days = today + timedelta(days=4)
four_days = four_days.date()
end_next_wk= end_of_this_week_saturday + timedelta(weeks=1)
end_next_wk = end_next_wk.date()
end_two_wk = end_of_this_week_saturday + timedelta(weeks=2)
end_two_wk = end_two_wk.date()

dates_names = ['today','End of This Wk','Four Days','End of Next Wk','End of 2 Wks','EoM','2 Months','3 Months','4 Months', '5 Months', '6 Months', 'Dec Expiry']
dates_values = [current_date, end_this_wk, four_days, end_next_wk, end_two_wk, end_of_month, two_months, three_months, four_months, five_months, six_months, dec_exp]

In [7]:
dates = dict()
for i in list(range(0,len(dates_names))):
    dates[dates_names[i]] = dates_values[i]

In [8]:
current_spot = positions['OTHER']
current_spot = current_spot[current_spot.Type=='Spot'].reset_index(drop=True)
current_spot = current_spot['Price'][0]

# OTHER Position Report
All PnL figures are in AUD

In [9]:
other = dict()
workbook = openpyxl.Workbook()

for i,d in enumerate(dates_values):
    other[d] = Position_Reporting(positions, 'OTHER', d).combine_frame()
    
    sheet_name = dates_names[i]
    sheet = workbook.create_sheet(title=sheet_name)
    
    for row in dataframe_to_rows(other[d][0], index=False, header=True):
        sheet.append(row)
    
workbook.save('Other PnL Output.xlsx')    

In [10]:
# data type; 0=PnL, 1=Delta, 2=Option Theta, 3=Option Vega
def generate_summary_plots(mkt_data, data_type):
    if data_type==0:
        colname='Total_Pnl'
    elif data_type==1:
        colname='Total_Delta'
    elif data_type==2:
        colname='Option_Theta'
    elif data_type==3:
        colname='Option_Vega'
    fig = go.Figure()
    for i in list(range(0, len(dates_names))):
        date_value = dates_values[i]
        date_name = dates_names[i]
        fig.add_trace(go.Scatter(x=mkt_data[date_value][data_type].Price, y=mkt_data[date_value][data_type][colname], name=date_name))
    return fig

In [11]:
#fig_d = generate_summary_plots(other, 1)
#fig_p = generate_summary_plots(other, 0)
#fig_t = generate_summary_plots(other,2)
#fig_v = generate_summary_plots(other,3)

In [12]:
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_d.data)):
#    fig.add_trace(fig_p.data[i], row=1, col=1)
#    fig.add_trace(fig_d.data[i], row=1, col=2)
#
#fig.update_layout(title={'text': 'OTHER Position (PnL and Delta)','x': 0.5,'xanchor': 'center'})
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="PnL", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Delta", row=1, col=2)
#
#fig.add_vline(x=current_spot, line_width=1, line_dash="dash", line_color="green")
#
#fig.show()

In [13]:
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_d.data)):
#    fig.add_trace(fig_t.data[i], row=1, col=1)
#    fig.add_trace(fig_v.data[i], row=1, col=2)
#
#fig.update_layout(title={'text': 'OTHER Position (Theta and Vega)','x': 0.5,'xanchor': 'center'})
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="Theta", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Vega", row=1, col=2)
#fig.show()

In [14]:
## USE THIS STYLE OF GRAPHING FOR TODAY DATE ENTRY ##
#position_functions = Position_Reporting(positions, 'OTHER', current_date)
#pnl, delta, theta, vega, value = position_functions.combine_frame()
#
#fig_d = go.Figure()
#
#fig_d.add_trace(go.Bar(x=delta.Price, y=delta.Spot, name='Spot', base=0))
#fig_d.add_trace(go.Bar(x=delta.Price, y=delta.Fwds, name='Fwds', base=0))
#fig_d.add_trace(go.Bar(x=delta.Price, y=delta.Options, name='Options', base=0))
#fig_d.add_trace(go.Scatter(x=delta.Price, y=delta.Total_Delta, mode='lines', name='Total_Delta'))
#
#
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig_p = go.Figure()
#
#fig_p.add_trace(go.Bar(x=pnl.Price, y=pnl.Spot, name='Spot', base=0))
#fig_p.add_trace(go.Bar(x=pnl.Price, y=pnl.Fwds, name='Fwds', base=0))
#fig_p.add_trace(go.Bar(x=pnl.Price, y=pnl.Options, name='Options', base=0))
#fig_p.add_trace(go.Scatter(x=pnl.Price, y=pnl.Total_Pnl, mode='lines', name='Total_PnL'))
#
#
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_d.data)):
#    fig.add_trace(fig_d.data[i], row=1, col=2)
#    fig.add_trace(fig_p.data[i], row=1, col=1)
#    
#fig.update_layout(title={'text': 'Todays OTHER Pos (PnL and Delta)','x': 0.5,'xanchor': 'center'}, barmode='stack')
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="Pnl", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Delta", row=1, col=2)    
#fig.show()  

### Positions

In [15]:
positions['OTHER']

,Name,Type,Subtype,Strike,Vol,Expiry,Qty,Price,Rate,FX
0,EUA Dec25,Spot,NaN,NaN,NaN,NaT,0,72.670000,0.0249,1.64012
1,EUA Mar25 75C,Option,Call,75.0,0.346185,2025-03-26,75000,2.013557,0.0249,1.64012
2,EUA Mar25 80C,Option,Call,80.0,0.340742,2025-03-26,250000,0.714685,0.0249,1.64012
3,EUA Mar25 85C,Option,Call,85.0,0.345433,2025-03-26,325000,0.226147,0.0249,1.64012
4,EUA Mar25 90C,Option,Call,90.0,0.358682,2025-03-26,350000,0.073714,0.0249,1.64012


In [16]:
# Options for timing are: [current_date, end_of_month, one_month, three_months, six_months, end_of_year, end_of_feb, march_expiry]

def pnl_delta_plot(timing):
    position_functions = Position_Reporting(positions, 'OTHER', timing)
    p,d,t,v, value = position_functions.options()
    fig_p = go.Figure()
    for i in list(p)[1:-1]:
        fig_p.add_trace(go.Scatter(x=p.Price, y=p[i], name=i))

    fig_p.add_trace(go.Scatter(x=p.Price, y=p.Option_Pnl, mode='lines', name='Total_PnL'))

    #fig_p.update_layout(title='Option PnL Today', xaxis_title='Price', yaxis_title='pnl', barmode='stack')
    #fig_p.show()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    fig_d = go.Figure()
    for i in list(d)[1:-1]:
        fig_d.add_trace(go.Scatter(x=d.Price, y=d[i], name=i))

    fig_d.add_trace(go.Scatter(x=d.Price, y=d.Option_Delta, mode='lines', name='Total_Delta'))

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    fig = make_subplots(rows=1, cols=2)
    for i in range(0,len(fig_p.data)):
        fig.add_trace(fig_p.data[i], row=1, col=1)
        fig.add_trace(fig_d.data[i], row=1, col=2)

    fig.update_layout(title={'text': 'Todays Option Pos (PnL and Delta)','x': 0.5,'xanchor': 'center'}, barmode='stack')

    fig.update_xaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Pnl", row=1, col=1)
    fig.update_xaxes(title_text="Price", row=1, col=2)
    fig.update_yaxes(title_text="Delta", row=1, col=2)    
    fig.show()  

## Current Date

In [17]:
pnl_delta_plot(current_date)

## End of This Week

In [18]:
pnl_delta_plot(end_this_wk)

## Four Days

In [19]:
pnl_delta_plot(four_days)

## End of Next Week

In [20]:
pnl_delta_plot(end_next_wk)

## End of 2 Weeks

In [21]:
pnl_delta_plot(end_two_wk)

## End of Month

In [22]:
pnl_delta_plot(end_of_month)

## End of Next Month

In [23]:
pnl_delta_plot(two_months)

## End of Year

In [24]:
pnl_delta_plot(dec_exp)

In [25]:
### THIS IS FOR PLOTTING THETA AND VEGA

#fig_t = go.Figure()
#for i in list(t)[1:-1]:
#    fig_t.add_trace(go.Bar(x=t.Price, y=t[i], name=i, base=0))
#
#fig_t.add_trace(go.Scatter(x=t.Price, y=t.Option_Theta, mode='lines', name='Total_Theta'))
#
#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig_v = go.Figure()
#for i in list(v)[1:-1]:
#    fig_v.add_trace(go.Bar(x=v.Price, y=v[i], name=i, base=0))
#
#fig_v.add_trace(go.Scatter(x=v.Price, y=v.Option_Vega, mode='lines', name='Total_Vega'))
#
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#fig = make_subplots(rows=1, cols=2)
#for i in range(0,len(fig_p.data)):
#    fig.add_trace(fig_t.data[i], row=1, col=1)
#    fig.add_trace(fig_v.data[i], row=1, col=2)
#   
#fig.update_layout(title={'text': 'Todays Option Pos (Theta and Vega)','x': 0.5,'xanchor': 'center'}, barmode='stack')
#
#fig.update_xaxes(title_text="Price", row=1, col=1)
#fig.update_yaxes(title_text="Theta", row=1, col=1)
#fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="Vega", row=1, col=2)    
#fig.show()  